In [ ]:
from dep_tools.searchers import PystacSearcher
from dep_tools.loaders import OdcLoader
from dep_tools.grids import PACIFIC_GRID_10

from dep_tools.s2_utils import mask_clouds

from odc.stac import configure_rio
import folium

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Set up rasterio
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})

In [ ]:
# Study site configuration
item_id = "63,20"  # West viti levu
datetime="2024-08-03"

# item_id = "56,49"  # Atol with cloud mask issues
# datetime = "2024-08-05"

# And get the study site
tile_index = tuple(int(i) for i in item_id.split(","))
geobox = PACIFIC_GRID_10.tile_geobox(tile_index)

# Load low-res if you want it faster
# geobox = geobox.crop(100)

geobox.explore()

In [ ]:
# Search for data
searcher = PystacSearcher(
    catalog="https://earth-search.aws.element84.com/v1",
    collections=["sentinel-2-c1-l2a"],
    datetime=datetime,   
)

items = searcher.search(area=geobox)
print(f"Found {len(items)} items")

In [ ]:
# Set up a data loader
loader = OdcLoader(
    bands=["scl", "red", "blue", "green", "cloud"],
    chunks=dict(time=1, x=3201, y=3201),
    groupby="solar_day",
    fail_on_error=False,
)

# Run the load process, which is lazy-loaded
data = loader.load(items, areas=geobox)
data

In [ ]:
data = data.squeeze().compute()

In [ ]:
data[["red", "green", "blue"]].to_array().plot.imshow(size=10, vmin=1000, vmax=3000)

In [ ]:
_, mask = mask_clouds(data, filters=[("dilation", 3), ("erosion", 2)], return_mask=True)
_, mask_new = mask_clouds(data, filters=[("erosion", 3), ("dilation", 6)], return_mask=True)
_, no_filter = mask_clouds(data, return_mask=True)

In [ ]:
m = folium.Map(location=geobox.geographic_extent.centroid.coords[0][::-1], zoom_start=10)

data.odc.to_rgba(vmin=1000, vmax=3000).odc.add_to(m, name="RGB")
mask.where(mask != 0).odc.add_to(m, name="Cloud Mask (old)", vmin=0, vmax=1)
mask_new.where(mask_new != 0).odc.add_to(m, name="Cloud Mask (new)", vmin=0, vmax=1)
no_filter.where(no_filter != 0).odc.add_to(m, name="Cloud Mask (no filter)", vmin=0, vmax=1)
data.cloud.odc.add_to(m, name="Cloud Probability")
data.scl.odc.add_to(m, name="SCL")

folium.LayerControl().add_to(m)

m

In [ ]:
m.save("cloudtesting_fiji.html")